In [2]:
!open .

In [86]:
# %load neo_quant.py
import pandas as pd
import numpy as np
import requests
import bs4
import time
import datetime
import matplotlib.pyplot as plt
from dateutil import parser
from matplotlib import font_manager, rc
from IPython.display import HTML
pd.set_option('display.max_colwidth', -1)
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
pd.options.display.max_rows = 60
%matplotlib inline

my_portfolio = ['동국제강', '포스코','현대건설','GS건설','한국조선해양','OCI','미래에셋대우','삼성증권','삼성중공업','키움증권','LG화학']
my_portfolio_code_list = ['A001230','A005490','A000720','A006360','A009540','A010060','A006800','A016360','A010140','A039490','A051910']

def make_code(x):
    x = str(x)
    return 'A' + '0' * (6-len(x)) + x


def make_code2(x):
    x = str(x)
    return '0' * (6-len(x)) + x


def get_all_company_info():
    kospi = pd.read_excel('data/kospi.xls')
    kosdaq = pd.read_excel('data/kosdaq.xls')
    kospi['구분'] = '코스피'
    kosdaq['구분'] = '코스닥'
    companies = pd.concat([kospi, kosdaq])
    companies = companies[['구분','종목코드','기업명']]
#     companies['종목코드'] = companies['종목코드'].apply(modifyCompanyCode)
    companies = companies.reset_index()
    return companies

def get_company_data(min_price=0):
    kospi = pd.read_excel('data/kospi.xls')
    kosdaq = pd.read_excel('data/kosdaq.xls')
    kospi['구분'] = '코스피'
    kosdaq['구분'] = '코스닥'
    companies = pd.concat([kospi, kosdaq])
    companies = companies[['종목코드','기업명','구분','업종코드','업종','액면가(원)','상장주식수(주)', '자본금(원)']]
    companies['상장주식수(주)'] = companies['상장주식수(주)'].str.replace(',','').astype(int)
    companies['자본금(원)'] = companies['자본금(원)'].str.replace(',','').astype(int)
    companies['액면가(원)'] = companies['액면가(원)'].str.replace(',','').astype(float)
    companies = companies[companies['액면가(원)'] >= min_price]
    companies['액면가(원)'] = companies['액면가(원)'].astype(int)
    companies = companies.set_index('종목코드')
    companies = company_apply_a_prefix(companies)
    return companies

def company_apply_a_prefix(companies):
    old_index = pd.Series(companies.index)
    new_index = old_index.map(make_code)
    return companies.set_index(new_index)

def company_apply_no_prefix(companies):
    old_index = pd.Series(companies.index)
    new_index = old_index.map(make_code2)
    return companies.set_index(new_index)

def remove_a_from_company_code(company_df):
    code_list = []
    for code in company_df.index:
        code = code.replace('A','')
        code_list.append(code)
    return company_df.set_index(pd.Series(code_list))

def get_price_data():
    price_path = r'data/price_data_2013.xlsx'
    price_df = pd.read_excel(price_path)
    price_df = price_df.set_index(price_df.columns[0])
#     price_df.columns = 'A' + price_df.columns 
    return price_df

def get_fs_data():
    fs_path = r'data/fs_data.xlsx'
    return get_finance_data(fs_path)

def get_fr_data():
    fr_path = r'data/fr_data.xlsx'
    return get_finance_data(fr_path)

def get_invest_data():
    invest_path = r'data/invest_data.xlsx'
    return get_finance_data(invest_path)

def load_all_data():
    companies = get_company_data()
    prices = get_price_data()
    fs_df = get_fs_data()
    fr_df = get_fr_data()
    iv_df = get_invest_data()
    return companies, prices, fs_df, fr_df, iv_df
    
# 액면가 1000원 이상 회사 리스트 가져와서 코드앞에 A 붙이기
# companies = get_company_info(1000)
# companies = apply_a_type_code(companies)

def filter_company(st_df, companies):
    new_company_list = []
    for num, code in enumerate(st_df.index):
        if code in companies.index:
            new_company_list.append(code)

    return st_df.loc[new_company_list]

def add_company_info(st_df, companies):
    return pd.merge(st_df, companies, how='inner', left_index=True, right_index=True)

# 데이터프레임에서 회사코드로 필터링한 후 회사 정보 추가하기
# roa_filter_info = add_company_info(filter_company(roa, companies), companies)

def get_kospi_list(st_df):
    return st_df[st_df['구분']=='코스피']

def get_kosdaq_list(st_df):
    return st_df[st_df['구분']=='코스닥']

def get_price_over_list(st_df, price):
    return st_df[st_df['액면가(원)'] >= price]

def get_company_code(name, company_df):
    return company_df[company_df['기업명']==name].index[0]

def get_company_code_list(company_name_list, company_df):
    code_list = []
    for company_name in company_name_list:
        for num, name in enumerate(company_df['기업명']):
            if company_name in name:
                code_list.append({'code':company_df.index[num], 'name':name})
    return code_list

def get_company_name(company_code, company_df):
    return company_df.loc[company_code]['기업명']

def show_chart(company_name_list, company_df, price_df, year_duration=1):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    company_list = get_company_code_list(company_name_list, company_df)
    if len(company_list) == 0:
        print('no company with name' + company_name)
        return
    code = company_list[0]['code']
    name = company_list[0]['name']
#     code = code.replace('A','')
    strategy_price = price_df[code][start_date:end_date]
    strategy_df = pd.DataFrame({'price':strategy_price})
    plt.figure(figsize=(10, 6))
    strategy_df['price'].plot(label=name)
    plt.legend()
    plt.show()          

def show_multi_chart(company_code_list, price_df, company_df, year_duration=1):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    strategy_price = price_df[company_code_list][start_date:end_date]
    num_row = int((len(company_code_list)-1)/2)+1
    plt.figure(figsize=(10, num_row*5))
    for i, code in enumerate(company_code_list):
        ax = plt.subplot(num_row, 4, i+1)
        name = get_company_name(code, company_df)
        ax.title.set_text(name)
        ax.plot(strategy_price.index, strategy_price[code])
    plt.show()
    
def show_detail_chart(company_name, company_df, price_df, year_duration=1):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    company_list = get_company_code_list(company_name, company_df)
    if len(company_list) == 0:
        print('no company with name' + company_name)
        return
    code = company_list[0]['code']
    name = company_list[0]['name']
#     code = code.replace('A','')
    strategy_price = price_df[code][start_date:end_date]
    strategy_df = pd.DataFrame({'price':strategy_price})
    strategy_df
    ma5 = strategy_df['price'].rolling(window=5).mean()
    strategy_df['ma5'] = ma5
    ma10 = strategy_df['price'].rolling(window=10).mean()
    strategy_df['ma10'] = ma10
    ma20 = strategy_df['price'].rolling(window=20).mean()
    strategy_df['ma20'] = ma20
    ma60 = strategy_df['price'].rolling(window=60).mean()
    strategy_df['ma60'] = ma60
    ma120 = strategy_df['price'].rolling(window=120).mean()
    strategy_df['ma120'] = ma120
    plt.figure(figsize=(20, 12))
    # strategy_df['price'].plot(label=name)
    plt.plot(strategy_df.index, strategy_df['price'], color='darkblue',linewidth=3.0)
    plt.plot(strategy_df.index, strategy_df['ma5'], color='red', label='ma5')
    plt.plot(strategy_df.index, strategy_df['ma10'], color='blue', label='ma10')
    plt.plot(strategy_df.index, strategy_df['ma20'], color='green', label='ma20')
    plt.plot(strategy_df.index, strategy_df['ma60'], color='cyan', label='ma60')
    plt.plot(strategy_df.index, strategy_df['ma120'], color='yellow', label='ma120')
    plt.title(name)
    plt.xlabel("duration")
    plt.ylabel("price")
    plt.legend(loc='upper right')
    plt.grid()
    plt.show() 
    
    
def get_maximum_earning_rate(price_df, company_df, year_duration=1, min_price=0, type='all'):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    strategy_price = price_df[start_date:end_date]
    strategy_price = strategy_price.fillna(method='bfill')
    last_price = strategy_price.iloc[-1]
    first_price = strategy_price.iloc[0]
    price_diff_df = pd.DataFrame({first_price.name:first_price, last_price.name:last_price})
#     price_diff_df.index = 'A' + price_diff_df.index
    price_diff_df['diff'] = price_diff_df[last_price.name] - price_diff_df[first_price.name]
    price_diff_df = price_diff_df[price_diff_df[last_price.name] > 5000]
    price_diff_df = price_diff_df[price_diff_df['diff'] > 0]
    # price_diff_df['ratio'] = price_diff_df['diff'] / price_diff_df[first_price.name]
    price_diff_df['ratio'] = ((price_diff_df[last_price.name] / price_diff_df[first_price.name]) - 1) * 100
    price_diff_df['ratio'] = price_diff_df['ratio'].astype(int)
    price_diff_df = price_diff_df.sort_values(by='ratio', ascending=False)
    price_diff_df = add_company_info(price_diff_df, company_df)
    if type == 'kospi':
        price_diff_df = get_kospi_list(price_diff_df)
    elif type == 'kosdaq':
        price_diff_df = get_kosdaq_list(price_diff_df)
    return price_diff_df
#     price_diff_df['fs_info'] = price_diff_df.index
#     price_diff_df['fs_info'] = price_diff_df['fs_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode={0}" target="_blank">fs</a>'.format(x))
#     price_diff_df['fr_info'] = price_diff_df.index
#     price_diff_df['fr_info'] = price_diff_df['fr_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode={0}" target="_blank">fr</a>'.format(x))
#     price_diff_df['iv_info'] = price_diff_df.index
#     price_diff_df['iv_info'] = price_diff_df['iv_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode={0}" target="_blank">iv</a>'.format(x))
#     return HTML(price_diff_df.to_html(escape=False))

def show_pf_earning_rate(code_list, price_df, year_duration=1, initial_monehy=100000000):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    st_backtest = backtest_with_code_list(price_df, code_list, start_date, end_date, initial_money)
    plt.figure(figsize=(10, 6))
    st_backtest['총변화율'].plot()
    plt.show()
    
def show_company_info(company_code_list, company_df):
    firm_df = company_df.loc[company_code_list]
    firm_df['fs_info'] = firm_df.index
    firm_df['fs_info'] = firm_df['fs_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode={0}" target="_blank">fs</a>'.format(x))
    firm_df['fr_info'] = firm_df.index
    firm_df['fr_info'] = firm_df['fr_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode={0}" target="_blank">fr</a>'.format(x))
    firm_df['iv_info'] = firm_df.index
    firm_df['iv_info'] = firm_df['iv_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode={0}" target="_blank">iv</a>'.format(x))
    return HTML(firm_df.to_html(escape=False))
    
def show_company_info_from_name(firm_name, company_df):
    company_list = get_company_code_list(firm_name, company_df)
    if len(company_list) == 0:
        print('no company with name' + company_name)
        return
    company_list
    code_list = []
    for company in company_list:
        code_list.append(company['code'])
    return show_company_info(code_list, companies)

def get_earning_rate(firm_name, company_df, price_df, year_duration=1):
    code_list = get_company_code_list(firm_name, company_df)
    if len(code_list) == 0:
        return "No Company with name : " + firm_name
    name = code_list[0]['name']
    code = code_list[0]['code']
#     code = code.replace('A','')
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    strategy_price = price_df[code][start_date:end_date]
    strategy_price = strategy_price.dropna()
    last_price = strategy_price.iloc[-1]
    first_price = strategy_price.iloc[0]
    print(str(strategy_price.index[0])+" : "+str(first_price))
    print(str(strategy_price.index[-1])+" : "+str(last_price))
    profit = int((last_price/first_price - 1) * 100)
    return name + " : " + str(profit) + '%'

def get_vaild_code_from_price_df(code_list, price_df):
    new_code_list = []
    for code in code_list:
        if code in price_df.columns:
            new_code_list.append(code)
    return new_code_list        

def show_business_trend(company_df, price_df, year_duration=1):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)

    company_df['업종코드'] = company_df['업종코드'].astype(str)
    company_df['업종구분'] = company_df['업종코드'] + ' (' + company_df['업종'] + ')'
    all_busi = company_df['업종구분'].unique()
    busi_list = []
    for busi_name in all_busi:
        temp_list = company_df[company_df['업종구분'] == busi_name]
        busi_list.append({'name':busi_name + ' - ' + str(len(temp_list)),'list':temp_list.index})

    for i, busi in enumerate(busi_list):
        busi_name = busi['name']
        busi_code_list = busi['list']
        strategy_price = price_df[get_vaild_code_from_price_df(busi_code_list, price_df)][start_date:end_date]
        strategy_price.fillna(method='bfill')
        busi_earning = strategy_price.sum(axis=1)
        temp_df = pd.DataFrame({busi_name:busi_earning})
        if i == 0:
            total_busi_earning = temp_df
        else:
            total_busi_earning = pd.merge(total_busi_earning, temp_df, how='outer', left_index=True, right_index=True)
    num_row = int((len(total_busi_earning.columns)-1)/4)+1
    plt.figure(figsize=(6*4, num_row*6))
    for i, busi in enumerate(total_busi_earning.columns):
        ax = plt.subplot(num_row, 4, i+1)
        ax.title.set_text(busi)
        ax.plot(total_busi_earning.index, total_busi_earning[busi])
    plt.show()    
    
def get_company_list_from_business_code(busi_code, company_df):
    return company_df[company_df['업종코드'] == busi_code].index

# [코드 3.15] 재무제표 데이터를 가져와 데이터프레임으로 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_fs_dataframe(firm_code):
    fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode=' + firm_code
    fs_page = requests.get(fs_url)
    fs_tables = pd.read_html(fs_page.text)
    
    temp_df = fs_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df[temp_df.columns[:4]]
    temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]

    temp_df2 = fs_tables[2]
    temp_df2 = temp_df2.set_index(temp_df2.columns[0])
    temp_df2 = temp_df2.loc[['자산', '부채', '자본']]

    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

    fs_df = pd.concat([temp_df, temp_df2, temp_df3])
    
    return fs_df

# [코드 3.19] 데이터프레임 형태 바꾸기 코드 함수화 (CH3. 데이터 수집하기.ipynb)

def change_df(firm_code, dataframe):
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code : dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]
        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)    
    
    return total_df

# [코드 3.21] 재무 비율 데이터프레임을 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_fr_dataframe(firm_code):
    fr_url = 'https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode=' + firm_code
    fr_page = requests.get(fr_url)
    fr_tables = pd.read_html(fr_page.text)
    
    temp_df = fr_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['유동비율계산에 참여한 계정 펼치기',
                           '부채비율계산에 참여한 계정 펼치기',
                           '영업이익증가율계산에 참여한 계정 펼치기',
                           'ROA계산에 참여한 계정 펼치기',
                           'ROIC계산에 참여한 계정 펼치기']]
    temp_df.index = ['유동비율', '부채비율', '영업이익률', 'ROA', 'ROIC']
    return temp_df

# [코드 3.23] 투자지표 데이터프레임을 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_invest_dataframe(firm_code):
    invest_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode=' + firm_code
    invest_page = requests.get(invest_url)
    invest_tables = pd.read_html(invest_page.text)
    temp_df = invest_tables[1]
    
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['PER계산에 참여한 계정 펼치기',
                       'PCR계산에 참여한 계정 펼치기',
                       'PSR계산에 참여한 계정 펼치기',
                       'PBR계산에 참여한 계정 펼치기',
                      '총현금흐름']]
    temp_df.index = ['PER', 'PCR', 'PSR', 'PBR', '총현금흐름']
    return temp_df

#  [코드 3.40] 가격을 가져와 데이터프레임 만드는 함수 (CH3. 데이터 수집하기 2.ipynb)

def make_price_dataframe(code, timeframe, count):
    url = 'https://fchart.stock.naver.com/sise.nhn?requestType=0'
    price_url = url + '&symbol=' + code + '&timeframe=' + timeframe + '&count=' + count
    price_data = requests.get(price_url)
    price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml')
    item_list = price_data_bs.find_all('item')
    
    date_list = []
    price_list = []
    for item in item_list:
        temp_data = item['data']
        datas = temp_data.split('|')
        date_list.append(datas[0])
        price_list.append(datas[4])

    price_df = pd.DataFrame({code:price_list}, index=date_list)
    
    return price_df

#  [코드 4.6] 재무 데이터 전처리하는 함수 (CH4. 전략 구현하기.ipynb)

def get_finance_data(path):
    data_path = path
    raw_data = pd.read_excel(data_path)
    raw_data = raw_data.set_index(raw_data.columns[0])
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])
    
    new_big_col = []
    for num, col in enumerate(big_col):
        if 'Unnamed' in col:
            new_big_col.append(new_big_col[num-1])
        else:
            new_big_col.append(big_col[num])
            
    raw_data.columns = [new_big_col, small_col]
    clean_df = raw_data.loc[ raw_data.index.dropna() ]
    
    return clean_df

def check_IFRS(x):
    if x == 'N/A(IFRS)':
        return np.NaN
    else:
        return x

#  [코드 4.14] PER기준으로 오름차순으로 정렬하여 주는 함수 (CH4. 전략 구현하기.ipynb)

def low_per(invest_df, index_date, num):
    invest_df[(index_date, 'PER')] = pd.to_numeric(invest_df[(index_date, 'PER')])
    per_sorted = invest_df.sort_values(by=(index_date, 'PER'))
    return per_sorted[index_date][:num]

#  [코드 4.15] ROA기준으로 내림차순으로 정렬하여 주는 함수 (CH4. 전략 구현하기.ipynb)

def high_roa(fr_df, index_date, num):
    fr_df[(index_date, 'ROA')] = fr_df[(index_date, 'ROA')].apply(check_IFRS)
    fr_df[(index_date, 'ROA')] = pd.to_numeric(fr_df[(index_date, 'ROA')] )
    sorted_roa = fr_df.sort_values(by=(index_date, 'ROA'), ascending=False)
    return sorted_roa[index_date][:num]


#  [코드 4.22] 마법공식 함수로 만들기 (CH4. 전략 구현하기.ipynb)

def magic_formula(fr_df, invest_df, index_date, num):
    per = low_per(invest_df, index_date, None)
    roa = high_roa(fr_df, index_date, None)
    per['per순위'] = per['PER'].rank()
    roa['roa순위'] = roa['ROA'].rank(ascending=False)
    magic = pd.merge(per, roa, how='outer', left_index=True, right_index=True)
    magic['마법공식 순위'] = (magic['per순위'] + magic['roa순위']).rank().sort_values()
    magic = magic.sort_values(by='마법공식 순위')
    return magic[:num]

#  [코드 4.23] 저평가 지수를 기준으로 정렬하여 순위 만들어 주는 함수 (CH4. 전략 구현하기.ipynb)

def get_value_rank(invest_df, value_type, index_date, num):
    invest_df[(index_date,  value_type)] = pd.to_numeric(invest_df[(index_date,  value_type)])
    value_sorted = invest_df.sort_values(by=(index_date,  value_type))[index_date]
    value_sorted[  value_type + '순위'] = value_sorted[value_type].rank()
    return value_sorted[[value_type, value_type + '순위']][:num]

#  [코드 4.25] 저평가 지표 조합 함수 (CH4. 전략 구현하기.ipynb)

def make_value_combo(value_list, invest_df, index_date, num):
    
    for i, value in enumerate(value_list):
        temp_df = get_value_rank(invest_df, value, index_date, None)
        if i == 0:
            value_combo_df = temp_df
            rank_combo = temp_df[value + '순위']
        else:
            value_combo_df = pd.merge(value_combo_df, temp_df, how='outer', left_index=True, right_index=True)
            rank_combo = rank_combo + temp_df[value + '순위']
    
    value_combo_df['종합순위'] = rank_combo.rank()
    value_combo_df = value_combo_df.sort_values(by='종합순위')
    
    return value_combo_df[:num]

#  [코드 4.29] F-score 함수(CH4. 전략 구현하기.ipynb)

def get_fscore(fs_df, index_date, num):
    fscore_df = fs_df[index_date]
    fscore_df['당기순이익점수'] = fscore_df['당기순이익'] > 0
    fscore_df['영업활동점수'] = fscore_df['영업활동으로인한현금흐름'] > 0
    fscore_df['더큰영업활동점수'] = fscore_df['영업활동으로인한현금흐름'] > fscore_df['당기순이익']
    fscore_df['종합점수'] = fscore_df[['당기순이익점수', '영업활동점수', '더큰영업활동점수']].sum(axis=1)
    fscore_df = fscore_df[fscore_df['종합점수'] == 3]
    return fscore_df[:num]

#  [코드 4.34] 모멘텀 데이터프레임 만들기 함수화 (CH4. 전략 구현하기.ipynb)

def get_momentum_rank(price_df, index_date, date_range, num):
    momentum_df = pd.DataFrame(price_df.pct_change(date_range).loc[index_date])
    momentum_df.columns = ['모멘텀']
    momentum_df['모멘텀순위'] = momentum_df['모멘텀'].rank(ascending=False)
    momentum_df = momentum_df.sort_values(by='모멘텀순위')
    return momentum_df[:num]

#  [코드 4.39] 저평가 + Fscore 함수화 (CH4. 전략 구현하기.ipynb)

def get_value_quality(invest_df, fs_df, index_date, num):
    value = make_value_combo(['PER', 'PBR', 'PSR', 'PCR'], invest_df, index_date, None)
    quality = get_fscore(fs_df, index_date, None)
    value_quality = pd.merge(value, quality, how='outer', left_index=True, right_index=True)
    value_quality_filtered = value_quality[value_quality['종합점수'] == 3]
    vq_df = value_quality_filtered.sort_values(by='종합순위')
    return vq_df[:num]

#  [코드 5.12] 백테스트 함수 버젼1 (Ch5. 백테스트.ipynb)


def backtest_beta(price_df, strategy_df, start_date, end_date, initial_money):

    code_list = []
    for code in strategy_df.index:
#         code_list.append(code.replace('A',''))
        code_list.append(code)

    strategy_price = price_df[code_list][start_date:end_date]

    pf_stock_num = {}
    stock_amount = 0
    stock_pf = 0
    each_money = initial_money / len(strategy_df)
    for code in strategy_price.columns:
        temp = int( each_money / strategy_price[code][0] )
        pf_stock_num[code] = temp
        stock_amount = stock_amount + temp * strategy_price[code][0]
        stock_pf = stock_pf + strategy_price[code] * pf_stock_num[code]

    cash_amount = initial_money - stock_amount

    backtest_df = pd.DataFrame({'주식포트폴리오':stock_pf})
    backtest_df['현금포트폴리오'] = [cash_amount] * len(backtest_df)
    backtest_df['종합포트폴리오'] = backtest_df['주식포트폴리오'] + backtest_df['현금포트폴리오']
    backtest_df['일변화율'] = backtest_df['종합포트폴리오'].pct_change()
    backtest_df['총변화율'] = backtest_df['종합포트폴리오']/initial_money - 1
    
    return backtest_df

def backtest_with_code_list(price_df, code_list_to_test, start_date, end_date, initial_money):

    code_list = []
    for code in code_list_to_test:
#         code_list.append(code.replace('A',''))
        code_list.append(code)

    strategy_price = price_df[code_list][start_date:end_date]

    pf_stock_num = {}
    stock_amount = 0
    stock_pf = 0
    each_money = initial_money / len(code_list)
    for code in strategy_price.columns:
        temp = int( each_money / strategy_price[code][0] )
        pf_stock_num[code] = temp
        stock_amount = stock_amount + temp * strategy_price[code][0]
        stock_pf = stock_pf + strategy_price[code] * pf_stock_num[code]

    cash_amount = initial_money - stock_amount

    backtest_df = pd.DataFrame({'주식포트폴리오':stock_pf})
    backtest_df['현금포트폴리오'] = [cash_amount] * len(backtest_df)
    backtest_df['종합포트폴리오'] = backtest_df['주식포트폴리오'] + backtest_df['현금포트폴리오']
    backtest_df['일변화율'] = backtest_df['종합포트폴리오'].pct_change()
    backtest_df['총변화율'] = backtest_df['종합포트폴리오']/initial_money - 1
    
    return backtest_df

#  [코드 5.16] 해당 날짜에 가격이 없으면 투자 관련 데이터에서 해당 종목 없애는 함수 (Ch5. 백테스트.ipynb)

def select_code_by_price(price_df, data_df, start_date):
    new_code_list = []
    for code in price_df[start_date].iloc[0].dropna().index:
        new_code_list.append(code)
        
    selected_df =  data_df.loc[new_code_list]
    return selected_df

# [코드 5.24] 백테스트 시작날짜가 주어지면 전략 기준 날짜를 계산하는 함수 (Ch5. 백테스트.ipynb)

def get_strategy_date(start_date):
    temp_year = int(start_date.split('-')[0])
    temp_month = start_date.split('-')[1]
    if temp_month in '1 2 3 4 5'.split(' '):
        strategy_date = str(temp_year - 2) + '/12'
    else:
        strategy_date = str(temp_year - 1) + '/12'
    return strategy_date


# [코드 5.32] 리밸런싱 백테스트 함수화 (Ch5. 백테스트.ipynb)

def backtest_re(strategy, start_date, end_date, initial_money, price_df, fr_df, fs_df, num, value_type=None, value_list=None, date_range=None):
    
    start_year = int(start_date.split('-')[0])
    end_year = int(end_date.split('-')[0])

    total_df = 0
    for temp in range(start_year, end_year):
        this_term_start = str(temp) + '-' + start_date.split('-')[1]
        this_term_end = str(temp+1) + '-' + start_date.split('-')[1]
        strategy_date = get_strategy_date(this_term_start)
        
        if strategy.__name__ == 'high_roa':
            st_df = strategy(select_code_by_price(price_df, fr_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'magic_formula':
            st_df = strategy(select_code_by_price(price_df, invest_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'get_value_rank':
            st_df = strategy(select_code_by_price(price_df, invest_df, this_term_start), value_type, strategy_date, num)
        elif strategy.__name__ == 'make_value_combo':
            st_df = strategy(value_list, select_code_by_price(price_df, invest_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'get_fscore':
            st_df = strategy(select_code_by_price(price_df, fs_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'get_momentum_rank':
            st_df = strategy(price_df, price_df[this_term_start].index[0] , date_range, num)
        elif strategy.__name__ == 'get_value_quality':
            st_df = strategy(select_code_by_price(price_df, invest_df, this_term_start), 
                             select_code_by_price(price_df, fs_df, this_term_start), strategy_date, num)
        
        backtest = backtest_beta(price_df, st_df, this_term_start, this_term_end, initial_money)
        temp_end = backtest[this_term_end].index[0]
        backtest = backtest[:temp_end]
        initial_money =  backtest['종합포트폴리오'][-1]
        if temp == start_year:
            total_df = backtest
        else:
            total_df = pd.concat([total_df[:-1], backtest])

    total_df ['일변화율'] = total_df ['종합포트폴리오'].pct_change()
    total_df ['총변화율'] = total_df ['종합포트폴리오']/ total_df ['종합포트폴리오'][0] - 1
    
    return total_df


# [코드 5.40] MDD 함수화 (Ch5. 백테스트.ipynb)

def get_mdd(back_test_df):
    max_list = [0]
    mdd_list = [0]

    for i in back_test_df.index[1:]:
        max_list.append(back_test_df['총변화율'][:i].max())
        if max_list[-1] > max_list[-2]:
            mdd_list.append(0)
        else:
            mdd_list.append(min(back_test_df['총변화율'][i] - max_list[-1], mdd_list[-1])   )

    back_test_df['max'] = max_list
    back_test_df['MDD'] = mdd_list
    
    return back_test_df































In [87]:
companies = get_company_data()
# companies = remove_a_from_company_code(companies)

In [88]:
get_company_code('GS', companies)

'A078930'

In [56]:
pd.options.display.max_rows = 60


In [89]:
prices = get_price_data()

In [100]:
def make_price_dataframe(code, timeframe, count):
    if code.startswith('A'):
        request_code = code.replace('A','')
    url = 'https://fchart.stock.naver.com/sise.nhn?requestType=0'
    price_url = url + '&symbol=' + request_code + '&timeframe=' + timeframe + '&count=' + count
    price_data = requests.get(price_url)
    price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml')
    item_list = price_data_bs.find_all('item')
    
    date_list = []
    price_list = []
    for item in item_list:
        temp_data = item['data']
        datas = temp_data.split('|')
        date_list.append(datas[0])
        price_list.append(datas[4])

    price_df = pd.DataFrame({code:price_list}, index=date_list)
    
    return price_df

In [103]:
prices

,A095570,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A200670,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2013-08-28,NaN,10850,38721,NaN,NaN,13281,194000,98540,46800,96500,...,NaN,18050,NaN,18360,NaN,2035,2025,NaN,5950,NaN
2013-08-29,NaN,11550,39205,NaN,NaN,13235,199500,98068,47600,99000,...,NaN,17850,NaN,18360,NaN,2340,2070,NaN,5980,NaN
2013-08-30,NaN,11900,40175,NaN,NaN,13281,190000,100426,48500,101000,...,NaN,18500,NaN,18647,NaN,2400,2060,NaN,5970,NaN
2013-09-02,NaN,11950,39932,NaN,NaN,13328,191000,99483,49600,99100,...,NaN,19600,NaN,19042,NaN,2400,2055,NaN,5970,NaN
2013-09-03,NaN,11700,39690,NaN,NaN,13934,190500,102312,49200,100000,...,NaN,19200,NaN,19042,NaN,2270,2138,NaN,5960,NaN
2013-09-04,NaN,11450,39157,NaN,NaN,13934,191000,101840,49400,99600,...,NaN,19350,NaN,18612,NaN,2610,2138,NaN,5940,NaN
2013-09-05,NaN,11500,38430,NaN,NaN,13934,191500,104669,52000,100000,...,NaN,18900,NaN,18826,NaN,3000,2130,NaN,5940,NaN
2013-09-06,NaN,11300,38769,NaN,NaN,13934,190500,104669,51700,99600,...,NaN,18450,NaN,18970,NaN,3095,2098,NaN,5930,NaN
2013-09-09,NaN,10950,38818,NaN,NaN,14074,190000,105612,52700,99600,...,NaN,18050,NaN,18934,NaN,3045,2118,NaN,5930,NaN


In [106]:
code = 'A095570'
price_df = make_price_dataframe(code, 'day', '10')
price_df.index = pd.to_datetime(price_df.index)

In [107]:
price_df

,A095570
2019-10-07,4290
2019-10-08,4230
2019-10-10,4230
2019-10-11,4235
2019-10-14,4220
2019-10-15,4215
2019-10-16,4235
2019-10-17,4280
2019-10-18,4325
2019-10-21,4235


In [118]:
price_df.dtypes

A095570    object
dtype: object

In [122]:
price_df['A095570'] = price_df['A095570'].astype(float)

In [114]:
price_df_origin = pd.DataFrame({'A095570':prices['A095570']}) 

In [125]:
price_df_origin

,A095570
Unnamed: 0,
2013-08-28,NaN
2013-08-29,NaN
2013-08-30,NaN
2013-09-02,NaN
2013-09-03,NaN
2013-09-04,NaN
2013-09-05,NaN
2013-09-06,NaN
2013-09-09,NaN


In [126]:
price_df


,A095570
2019-10-07,4290.0
2019-10-08,4230.0
2019-10-10,4230.0
2019-10-11,4235.0
2019-10-14,4220.0
2019-10-15,4215.0
2019-10-16,4235.0
2019-10-17,4280.0
2019-10-18,4325.0
2019-10-21,4235.0


In [139]:
new_companies = pd.concat([price_df_origin, price_df])

In [140]:
new_companies = new_companies.loc[~new_companies.index.duplicated(keep='first')]


,A095570
2013-08-28,NaN
2013-08-29,NaN
2013-08-30,NaN
2013-09-02,NaN
2013-09-03,NaN
2013-09-04,NaN
2013-09-05,NaN
2013-09-06,NaN
2013-09-09,NaN
2013-09-10,NaN


In [158]:
prices.drop('A095570', axis=1)
prices_drop = prices.drop('A095570', axis=1)
prices_drop
price_df
price_merge = pd.merge(prices_drop, price_df, left_index=True, right_index=True, how='outer')

,A095570
2019-10-07,4290.0
2019-10-08,4230.0
2019-10-10,4230.0
2019-10-11,4235.0
2019-10-14,4220.0
2019-10-15,4215.0
2019-10-16,4235.0
2019-10-17,4280.0
2019-10-18,4325.0
2019-10-21,4235.0


In [159]:
price_merge - prices

,A000020,A000040,A000050,A000060,A000070,A000080,A000100,A000120,A000140,A000150,...,A900290,A900300,A900310,A900340,A950110,A950130,A950140,A950160,A950170,A950180
2013-08-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2013-08-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2013-08-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2013-09-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2013-09-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2013-09-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2013-09-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2013-09-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2013-09-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2013-09-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [157]:
price_merge

,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,A011150,...,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490,A095570
2013-08-28,10850.0,38721.0,NaN,NaN,13281.0,194000.0,98540.0,46800.0,96500.0,2590.0,...,18050.0,NaN,18360.0,NaN,2035.0,2025.0,NaN,5950.0,NaN,NaN
2013-08-29,11550.0,39205.0,NaN,NaN,13235.0,199500.0,98068.0,47600.0,99000.0,2595.0,...,17850.0,NaN,18360.0,NaN,2340.0,2070.0,NaN,5980.0,NaN,NaN
2013-08-30,11900.0,40175.0,NaN,NaN,13281.0,190000.0,100426.0,48500.0,101000.0,2595.0,...,18500.0,NaN,18647.0,NaN,2400.0,2060.0,NaN,5970.0,NaN,NaN
2013-09-02,11950.0,39932.0,NaN,NaN,13328.0,191000.0,99483.0,49600.0,99100.0,2620.0,...,19600.0,NaN,19042.0,NaN,2400.0,2055.0,NaN,5970.0,NaN,NaN
2013-09-03,11700.0,39690.0,NaN,NaN,13934.0,190500.0,102312.0,49200.0,100000.0,2635.0,...,19200.0,NaN,19042.0,NaN,2270.0,2138.0,NaN,5960.0,NaN,NaN
2013-09-04,11450.0,39157.0,NaN,NaN,13934.0,191000.0,101840.0,49400.0,99600.0,2630.0,...,19350.0,NaN,18612.0,NaN,2610.0,2138.0,NaN,5940.0,NaN,NaN
2013-09-05,11500.0,38430.0,NaN,NaN,13934.0,191500.0,104669.0,52000.0,100000.0,2625.0,...,18900.0,NaN,18826.0,NaN,3000.0,2130.0,NaN,5940.0,NaN,NaN
2013-09-06,11300.0,38769.0,NaN,NaN,13934.0,190500.0,104669.0,51700.0,99600.0,2640.0,...,18450.0,NaN,18970.0,NaN,3095.0,2098.0,NaN,5930.0,NaN,NaN
2013-09-09,10950.0,38818.0,NaN,NaN,14074.0,190000.0,105612.0,52700.0,99600.0,2600.0,...,18050.0,NaN,18934.0,NaN,3045.0,2118.0,NaN,5930.0,NaN,NaN
2013-09-10,11200.0,37945.0,NaN,NaN,14213.0,188000.0,103255.0,50500.0,99400.0,2620.0,...,18300.0,NaN,18862.0,NaN,2685.0,2118.0,NaN,5900.0,NaN,NaN


In [143]:
prices['A095570'] = new_companies

In [144]:
prices

,A095570,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A200670,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2013-08-28,NaN,10850,38721,NaN,NaN,13281,194000,98540,46800,96500,...,NaN,18050,NaN,18360,NaN,2035,2025,NaN,5950,NaN
2013-08-29,NaN,11550,39205,NaN,NaN,13235,199500,98068,47600,99000,...,NaN,17850,NaN,18360,NaN,2340,2070,NaN,5980,NaN
2013-08-30,NaN,11900,40175,NaN,NaN,13281,190000,100426,48500,101000,...,NaN,18500,NaN,18647,NaN,2400,2060,NaN,5970,NaN
2013-09-02,NaN,11950,39932,NaN,NaN,13328,191000,99483,49600,99100,...,NaN,19600,NaN,19042,NaN,2400,2055,NaN,5970,NaN
2013-09-03,NaN,11700,39690,NaN,NaN,13934,190500,102312,49200,100000,...,NaN,19200,NaN,19042,NaN,2270,2138,NaN,5960,NaN
2013-09-04,NaN,11450,39157,NaN,NaN,13934,191000,101840,49400,99600,...,NaN,19350,NaN,18612,NaN,2610,2138,NaN,5940,NaN
2013-09-05,NaN,11500,38430,NaN,NaN,13934,191500,104669,52000,100000,...,NaN,18900,NaN,18826,NaN,3000,2130,NaN,5940,NaN
2013-09-06,NaN,11300,38769,NaN,NaN,13934,190500,104669,51700,99600,...,NaN,18450,NaN,18970,NaN,3095,2098,NaN,5930,NaN
2013-09-09,NaN,10950,38818,NaN,NaN,14074,190000,105612,52700,99600,...,NaN,18050,NaN,18934,NaN,3045,2118,NaN,5930,NaN


In [137]:
new_companies = price_df_origin.merge(price_df,how='left',on=['A095570','A095570'], left_index=True, right_index=True)

In [141]:
new_companies = new_companies.loc[~new_companies.index.duplicated(keep='first')]

In [142]:
new_companies


,A095570
2013-08-28,NaN
2013-08-29,NaN
2013-08-30,NaN
2013-09-02,NaN
2013-09-03,NaN
2013-09-04,NaN
2013-09-05,NaN
2013-09-06,NaN
2013-09-09,NaN
2013-09-10,NaN


In [146]:
pri

,기업명,구분,업종코드,업종,액면가(원),상장주식수(주),자본금(원)
종목코드,,,,,,,
A095570,AJ네트웍스,코스피,147603,산업용 기계 및 장비 임대업,1000,46822295,46822295000
A068400,AJ렌터카,코스피,147601,운송장비 임대업,500,22146300,11073150000
A006840,AK홀딩스,코스피,116409,기타 금융업,5000,13247561,66237805000
A027410,BGF,코스피,116409,기타 금융업,1000,95716791,95716791000
A282330,BGF리테일,코스피,74701,종합 소매업,1000,17283906,17283906000
A138930,BNK금융지주,코스피,116409,기타 금융업,5000,325935246,1629676230000
A001460,BYC,코스피,31401,봉제의복 제조업,5000,624615,4200000000
A001040,CJ,코스피,116409,기타 금융업,5000,29176998,179047370000
A079160,CJ CGV,코스피,105901,"영화, 비디오물, 방송프로그램 제작 및 배급업",500,21161313,10580656500


In [145]:
new_companies


,A095570
2013-08-28,NaN
2013-08-29,NaN
2013-08-30,NaN
2013-09-02,NaN
2013-09-03,NaN
2013-09-04,NaN
2013-09-05,NaN
2013-09-06,NaN
2013-09-09,NaN
2013-09-10,NaN


In [110]:
new_companies = pd.merge(prices, price_df, left_index=True, right_index=True, how='outer')

In [111]:
new_companies

,A095570_x,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490,A095570_y
2013-08-28,NaN,10850.0,38721.0,NaN,NaN,13281.0,194000.0,98540.0,46800.0,96500.0,...,18050.0,NaN,18360.0,NaN,2035.0,2025.0,NaN,5950.0,NaN,NaN
2013-08-29,NaN,11550.0,39205.0,NaN,NaN,13235.0,199500.0,98068.0,47600.0,99000.0,...,17850.0,NaN,18360.0,NaN,2340.0,2070.0,NaN,5980.0,NaN,NaN
2013-08-30,NaN,11900.0,40175.0,NaN,NaN,13281.0,190000.0,100426.0,48500.0,101000.0,...,18500.0,NaN,18647.0,NaN,2400.0,2060.0,NaN,5970.0,NaN,NaN
2013-09-02,NaN,11950.0,39932.0,NaN,NaN,13328.0,191000.0,99483.0,49600.0,99100.0,...,19600.0,NaN,19042.0,NaN,2400.0,2055.0,NaN,5970.0,NaN,NaN
2013-09-03,NaN,11700.0,39690.0,NaN,NaN,13934.0,190500.0,102312.0,49200.0,100000.0,...,19200.0,NaN,19042.0,NaN,2270.0,2138.0,NaN,5960.0,NaN,NaN
2013-09-04,NaN,11450.0,39157.0,NaN,NaN,13934.0,191000.0,101840.0,49400.0,99600.0,...,19350.0,NaN,18612.0,NaN,2610.0,2138.0,NaN,5940.0,NaN,NaN
2013-09-05,NaN,11500.0,38430.0,NaN,NaN,13934.0,191500.0,104669.0,52000.0,100000.0,...,18900.0,NaN,18826.0,NaN,3000.0,2130.0,NaN,5940.0,NaN,NaN
2013-09-06,NaN,11300.0,38769.0,NaN,NaN,13934.0,190500.0,104669.0,51700.0,99600.0,...,18450.0,NaN,18970.0,NaN,3095.0,2098.0,NaN,5930.0,NaN,NaN
2013-09-09,NaN,10950.0,38818.0,NaN,NaN,14074.0,190000.0,105612.0,52700.0,99600.0,...,18050.0,NaN,18934.0,NaN,3045.0,2118.0,NaN,5930.0,NaN,NaN
2013-09-10,NaN,11200.0,37945.0,NaN,NaN,14213.0,188000.0,103255.0,50500.0,99400.0,...,18300.0,NaN,18862.0,NaN,2685.0,2118.0,NaN,5900.0,NaN,NaN


In [ ]:
for num, code in enumerate(companies['종목코드']):
    try:
        print(num, code)
        time.sleep(1)
        try:
            price_df = make_price_dataframe(code, 'day', '1500')
        except requests.exceptions.Timeout:
            time.sleep(60)
            price_df = make_price_dataframe(code, 'day', '1500')
        if num == 0 :
            total_price = price_df
        else:
            total_price = pd.merge(total_price, price_df, how='outer', right_index=True, left_index=True)
    except ValueError:
        continue
    except KeyError:
        continue

total_price

In [ ]:
origin_prices = prices
for num, code in enumerate(companies['종목코드']):
    try:
        print(num, code)
        time.sleep(1)
        try:
            price_df = make_price_dataframe(code, 'day', '1500')
        except requests.exceptions.Timeout:
            time.sleep(60)
            price_df = make_price_dataframe(code, 'day', '1500')
        if num == 0 :
            total_price = price_df
        else:
            total_price = pd.merge(total_price, price_df, how='outer', right_index=True, left_index=True)
    except ValueError:
        continue
    except KeyError:
        continue


In [169]:
prices = get_price_data()

In [174]:
prices.index.name = ''

In [175]:
prices

,A095570,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A200670,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490
,,,,,,,,,,,,,,,,,,,,,
2013-08-28,NaN,10850,38721,NaN,NaN,13281,194000,98540,46800,96500,...,NaN,18050,NaN,18360,NaN,2035,2025,NaN,5950,NaN
2013-08-29,NaN,11550,39205,NaN,NaN,13235,199500,98068,47600,99000,...,NaN,17850,NaN,18360,NaN,2340,2070,NaN,5980,NaN
2013-08-30,NaN,11900,40175,NaN,NaN,13281,190000,100426,48500,101000,...,NaN,18500,NaN,18647,NaN,2400,2060,NaN,5970,NaN
2013-09-02,NaN,11950,39932,NaN,NaN,13328,191000,99483,49600,99100,...,NaN,19600,NaN,19042,NaN,2400,2055,NaN,5970,NaN
2013-09-03,NaN,11700,39690,NaN,NaN,13934,190500,102312,49200,100000,...,NaN,19200,NaN,19042,NaN,2270,2138,NaN,5960,NaN
2013-09-04,NaN,11450,39157,NaN,NaN,13934,191000,101840,49400,99600,...,NaN,19350,NaN,18612,NaN,2610,2138,NaN,5940,NaN
2013-09-05,NaN,11500,38430,NaN,NaN,13934,191500,104669,52000,100000,...,NaN,18900,NaN,18826,NaN,3000,2130,NaN,5940,NaN
2013-09-06,NaN,11300,38769,NaN,NaN,13934,190500,104669,51700,99600,...,NaN,18450,NaN,18970,NaN,3095,2098,NaN,5930,NaN
2013-09-09,NaN,10950,38818,NaN,NaN,14074,190000,105612,52700,99600,...,NaN,18050,NaN,18934,NaN,3045,2118,NaN,5930,NaN


In [188]:
def update_origin_price_df(origin_price_df, code, count):
    origin_prices = origin_price_df
    price_df = make_price_dataframe(code, 'day', count)
    price_df.index = pd.to_datetime(price_df.index)
    price_df[code] = price_df[code].astype(float)
    price_df_origin = pd.DataFrame({code:origin_prices[code]}) 
    new_price_df = pd.concat([price_df_origin, price_df])
    new_price_df = new_price_df.loc[~new_price_df.index.duplicated(keep='last')]
    origin_prices = origin_prices.drop(code, axis=1)
    origin_prices = pd.merge(origin_prices, new_price_df, left_index=True, right_index=True, how='outer')
    return origin_prices

# update_origin_price_df(prices, 'A095570', 10)

In [189]:
update_prices = update_origin_price_df(prices, 'A095570', '10')

In [190]:
update_prices


,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,A011150,...,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490,A095570
2013-08-28,10850.0,38721.0,NaN,NaN,13281.0,194000.0,98540.0,46800.0,96500.0,2590.0,...,18050.0,NaN,18360.0,NaN,2035.0,2025.0,NaN,5950.0,NaN,NaN
2013-08-29,11550.0,39205.0,NaN,NaN,13235.0,199500.0,98068.0,47600.0,99000.0,2595.0,...,17850.0,NaN,18360.0,NaN,2340.0,2070.0,NaN,5980.0,NaN,NaN
2013-08-30,11900.0,40175.0,NaN,NaN,13281.0,190000.0,100426.0,48500.0,101000.0,2595.0,...,18500.0,NaN,18647.0,NaN,2400.0,2060.0,NaN,5970.0,NaN,NaN
2013-09-02,11950.0,39932.0,NaN,NaN,13328.0,191000.0,99483.0,49600.0,99100.0,2620.0,...,19600.0,NaN,19042.0,NaN,2400.0,2055.0,NaN,5970.0,NaN,NaN
2013-09-03,11700.0,39690.0,NaN,NaN,13934.0,190500.0,102312.0,49200.0,100000.0,2635.0,...,19200.0,NaN,19042.0,NaN,2270.0,2138.0,NaN,5960.0,NaN,NaN
2013-09-04,11450.0,39157.0,NaN,NaN,13934.0,191000.0,101840.0,49400.0,99600.0,2630.0,...,19350.0,NaN,18612.0,NaN,2610.0,2138.0,NaN,5940.0,NaN,NaN
2013-09-05,11500.0,38430.0,NaN,NaN,13934.0,191500.0,104669.0,52000.0,100000.0,2625.0,...,18900.0,NaN,18826.0,NaN,3000.0,2130.0,NaN,5940.0,NaN,NaN
2013-09-06,11300.0,38769.0,NaN,NaN,13934.0,190500.0,104669.0,51700.0,99600.0,2640.0,...,18450.0,NaN,18970.0,NaN,3095.0,2098.0,NaN,5930.0,NaN,NaN
2013-09-09,10950.0,38818.0,NaN,NaN,14074.0,190000.0,105612.0,52700.0,99600.0,2600.0,...,18050.0,NaN,18934.0,NaN,3045.0,2118.0,NaN,5930.0,NaN,NaN
2013-09-10,11200.0,37945.0,NaN,NaN,14213.0,188000.0,103255.0,50500.0,99400.0,2620.0,...,18300.0,NaN,18862.0,NaN,2685.0,2118.0,NaN,5900.0,NaN,NaN


In [191]:
update_prices = update_origin_price_df(update_prices, 'A068400', '10')

In [192]:
update_prices

,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,A011150,A097950,...,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490,A095570,A068400
2013-08-28,38721.0,NaN,NaN,13281.0,194000.0,98540.0,46800.0,96500.0,2590.0,245000.0,...,NaN,18360.0,NaN,2035.0,2025.0,NaN,5950.0,NaN,NaN,10850.0
2013-08-29,39205.0,NaN,NaN,13235.0,199500.0,98068.0,47600.0,99000.0,2595.0,250000.0,...,NaN,18360.0,NaN,2340.0,2070.0,NaN,5980.0,NaN,NaN,11550.0
2013-08-30,40175.0,NaN,NaN,13281.0,190000.0,100426.0,48500.0,101000.0,2595.0,254500.0,...,NaN,18647.0,NaN,2400.0,2060.0,NaN,5970.0,NaN,NaN,11900.0
2013-09-02,39932.0,NaN,NaN,13328.0,191000.0,99483.0,49600.0,99100.0,2620.0,256000.0,...,NaN,19042.0,NaN,2400.0,2055.0,NaN,5970.0,NaN,NaN,11950.0
2013-09-03,39690.0,NaN,NaN,13934.0,190500.0,102312.0,49200.0,100000.0,2635.0,263500.0,...,NaN,19042.0,NaN,2270.0,2138.0,NaN,5960.0,NaN,NaN,11700.0
2013-09-04,39157.0,NaN,NaN,13934.0,191000.0,101840.0,49400.0,99600.0,2630.0,265000.0,...,NaN,18612.0,NaN,2610.0,2138.0,NaN,5940.0,NaN,NaN,11450.0
2013-09-05,38430.0,NaN,NaN,13934.0,191500.0,104669.0,52000.0,100000.0,2625.0,271000.0,...,NaN,18826.0,NaN,3000.0,2130.0,NaN,5940.0,NaN,NaN,11500.0
2013-09-06,38769.0,NaN,NaN,13934.0,190500.0,104669.0,51700.0,99600.0,2640.0,268000.0,...,NaN,18970.0,NaN,3095.0,2098.0,NaN,5930.0,NaN,NaN,11300.0
2013-09-09,38818.0,NaN,NaN,14074.0,190000.0,105612.0,52700.0,99600.0,2600.0,272500.0,...,NaN,18934.0,NaN,3045.0,2118.0,NaN,5930.0,NaN,NaN,10950.0
2013-09-10,37945.0,NaN,NaN,14213.0,188000.0,103255.0,50500.0,99400.0,2620.0,273500.0,...,NaN,18862.0,NaN,2685.0,2118.0,NaN,5900.0,NaN,NaN,11200.0


In [195]:
prices_update = update_prices(companies, prices, '10') 

0 A095570
1 A068400
2 A006840
3 A027410
4 A282330
5 A138930
6 A001460
7 A001040
8 A079160
9 A000120
10 A011150
11 A097950
12 A037560
13 A000590
14 A012030
15 A016610
16 A005830
17 A000990
18 A139130
19 A004840
20 A155660
21 A069730
22 A017940
23 A007700
24 A114090
25 A078930
26 A006360
27 A001250
28 A007070
29 A012630
30 A039570
31 A089470
32 A294870
33 A082740
34 A175330
35 A234080
36 A001060
37 A096760
38 A105560
39 A009440
40 A119650
41 A092220
42 A001390
43 A001940
44 A025000
45 A092230
46 A000040
47 A044450
48 A030210
49 A058850
50 A058860
51 A093050
52 A003550
53 A034220
54 A001120
55 A051900
56 A032640
57 A011070
58 A066570
59 A108670
60 A051910
61 A079550
62 A006260
63 A000680
64 A010120
65 A229640
66 A023150
67 A035420
68 A005940
69 A034310
70 A008260
71 A010060
72 A100840
73 A064960
74 A003570
75 A036530
76 A010950
77 A034120
78 A101060
79 A004060
80 A001380
81 A002360
82 A009160
83 A034730
84 A011790
85 A018670
86 A001740
87 A006120
88 A210980
89 A096770
90 A001510
91 A28513

692 A008930
693 A128940
694 A009240
695 A020000
696 A003680
697 A105630
698 A069640
699 A016450
700 A009180
701 A213500
702 A014680
703 A004710
704 A010420
705 A004150
706 A025750
707 A004960
708 A011700
709 A001750
710 A018880
711 A009420
712 A014130
713 A300720
714 A002220
715 A006390
716 A003300
717 A051600
718 A052690
719 A130660
720 A002320
721 A097230
722 A003480
723 A180640
724 A005110
725 A009460
726 A000880
727 A027390
728 A088350
729 A000370
730 A012450
731 A009830
732 A003530
733 A195870
734 A101530
735 A143210
736 A000720
737 A267270
738 A005440
739 A086280
740 A064350
741 A079430
742 A012330
743 A010620
744 A069960
745 A004560
746 A011200
747 A004310
748 A126560
749 A017800
750 A307950
751 A011210
752 A267260
753 A005380
754 A004020
755 A011760
756 A267250
757 A001500
758 A227840
759 A001450
760 A057050
761 A093240
762 A003010
763 A111110
764 A008770
765 A002460
766 A013520
767 A241590
768 A006060
769 A010690
770 A133820
771 A010660
772 A000850
773 A016580
774 A032560
775 

1346 A065350
1347 A056000
1348 A002800
1349 A307180
1350 A323280
1351 A017000
1352 A012790
1353 A138070
1354 A277480
1355 A323230
1356 A056700
1357 A187270
1358 A243840
1359 A108320
1360 A222800
1361 A036710
1362 A160980
1363 A217330
1364 A010280
1365 A122800
1366 A208640
1367 A222420
1368 A037760
1369 A136510
1370 A099320
1371 A049960
1372 A050890
1373 A066790
1374 A222080
1375 A264660
1376 A115530
1377 A225330
1378 A115480
1379 A096530
1380 A900120
1381 A101240
1382 A047920
1383 A060590
1384 A036170
1385 A260930
1386 A013990
1387 A123860
1388 A025980
1389 A058220
1390 A125210
1391 A052710
1392 A074430
1393 A092040
1394 A083930
1395 A149950
1396 A036010
1397 A050860
1398 A246720
1399 A067390
1400 A127710
1401 A154030
1402 A227610
1403 A143160
1404 A054800
1405 A099190
1406 A289010
1407 A214430
1408 A040910
1409 A068940
1410 A052860
1411 A069920
1412 A038880
1413 A123010
1414 A101390
1415 A226350
1416 A078860
1417 A114810
1418 A031310
1419 A185490
1420 A059100
1421 A175250
1422 A052460

1977 A214450
1978 A208340
1979 A177830
1980 A037070
1981 A150900
1982 A037030
1983 A047310
1984 A170790
1985 A123260
1986 A049120
1987 A038950
1988 A106240
1989 A131760
1990 A065690
1991 A140860
1992 A091700
1993 A194510
1994 A038160
1995 A032800
1996 A043090
1997 A027710
1998 A225590
1999 A054300
2000 A068050
2001 A222110
2002 A263750
2003 A251970
2004 A087010
2005 A043370
2006 A119500
2007 A016670
2008 A056730
2009 A022100
2010 A009520
2011 A189690
2012 A256630
2013 A141020
2014 A290720
2015 A005670
2016 A094940
2017 A007330
2018 A093380
2019 A023900
2020 A195440
2021 A214270
2022 A220100
2023 A035200
2024 A203690
2025 A053610
2026 A053160
2027 A075130
2028 A023770
2029 A300080
2030 A032580
2031 A051380
2032 A241820
2033 A237750
2034 A131390
2035 A054340
2036 A024850
2037 A319660
2038 A031980
2039 A002230
2040 A137400
2041 A128660
2042 A006140
2043 A087600
2044 A057880
2045 A064800
2046 A161580
2047 A270520
2048 A284620
2049 A320000
2050 A261200
2051 A067310
2052 A307160
2053 A166090

In [196]:
prices_update

,A095570,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A200670,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490
2013-08-28,NaN,10850.0,38721.0,NaN,NaN,13281.0,194000.0,98540.0,46800.0,96500.0,...,NaN,18050.0,NaN,18360.0,NaN,2035.0,2025.0,NaN,5950.0,NaN
2013-08-29,NaN,11550.0,39205.0,NaN,NaN,13235.0,199500.0,98068.0,47600.0,99000.0,...,NaN,17850.0,NaN,18360.0,NaN,2340.0,2070.0,NaN,5980.0,NaN
2013-08-30,NaN,11900.0,40175.0,NaN,NaN,13281.0,190000.0,100426.0,48500.0,101000.0,...,NaN,18500.0,NaN,18647.0,NaN,2400.0,2060.0,NaN,5970.0,NaN
2013-09-02,NaN,11950.0,39932.0,NaN,NaN,13328.0,191000.0,99483.0,49600.0,99100.0,...,NaN,19600.0,NaN,19042.0,NaN,2400.0,2055.0,NaN,5970.0,NaN
2013-09-03,NaN,11700.0,39690.0,NaN,NaN,13934.0,190500.0,102312.0,49200.0,100000.0,...,NaN,19200.0,NaN,19042.0,NaN,2270.0,2138.0,NaN,5960.0,NaN
2013-09-04,NaN,11450.0,39157.0,NaN,NaN,13934.0,191000.0,101840.0,49400.0,99600.0,...,NaN,19350.0,NaN,18612.0,NaN,2610.0,2138.0,NaN,5940.0,NaN
2013-09-05,NaN,11500.0,38430.0,NaN,NaN,13934.0,191500.0,104669.0,52000.0,100000.0,...,NaN,18900.0,NaN,18826.0,NaN,3000.0,2130.0,NaN,5940.0,NaN
2013-09-06,NaN,11300.0,38769.0,NaN,NaN,13934.0,190500.0,104669.0,51700.0,99600.0,...,NaN,18450.0,NaN,18970.0,NaN,3095.0,2098.0,NaN,5930.0,NaN
2013-09-09,NaN,10950.0,38818.0,NaN,NaN,14074.0,190000.0,105612.0,52700.0,99600.0,...,NaN,18050.0,NaN,18934.0,NaN,3045.0,2118.0,NaN,5930.0,NaN
2013-09-10,NaN,11200.0,37945.0,NaN,NaN,14213.0,188000.0,103255.0,50500.0,99400.0,...,NaN,18300.0,NaN,18862.0,NaN,2685.0,2118.0,NaN,5900.0,NaN


In [197]:
prices

,A095570,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A200670,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490
,,,,,,,,,,,,,,,,,,,,,
2013-08-28,NaN,10850,38721,NaN,NaN,13281,194000,98540,46800,96500,...,NaN,18050,NaN,18360,NaN,2035,2025,NaN,5950,NaN
2013-08-29,NaN,11550,39205,NaN,NaN,13235,199500,98068,47600,99000,...,NaN,17850,NaN,18360,NaN,2340,2070,NaN,5980,NaN
2013-08-30,NaN,11900,40175,NaN,NaN,13281,190000,100426,48500,101000,...,NaN,18500,NaN,18647,NaN,2400,2060,NaN,5970,NaN
2013-09-02,NaN,11950,39932,NaN,NaN,13328,191000,99483,49600,99100,...,NaN,19600,NaN,19042,NaN,2400,2055,NaN,5970,NaN
2013-09-03,NaN,11700,39690,NaN,NaN,13934,190500,102312,49200,100000,...,NaN,19200,NaN,19042,NaN,2270,2138,NaN,5960,NaN
2013-09-04,NaN,11450,39157,NaN,NaN,13934,191000,101840,49400,99600,...,NaN,19350,NaN,18612,NaN,2610,2138,NaN,5940,NaN
2013-09-05,NaN,11500,38430,NaN,NaN,13934,191500,104669,52000,100000,...,NaN,18900,NaN,18826,NaN,3000,2130,NaN,5940,NaN
2013-09-06,NaN,11300,38769,NaN,NaN,13934,190500,104669,51700,99600,...,NaN,18450,NaN,18970,NaN,3095,2098,NaN,5930,NaN
2013-09-09,NaN,10950,38818,NaN,NaN,14074,190000,105612,52700,99600,...,NaN,18050,NaN,18934,NaN,3045,2118,NaN,5930,NaN


In [198]:
prices_update.index = pd.to_datetime(prices_update.index)
prices_update.to_excel(r'data/price_data_2013_update.xlsx') 

In [199]:
prices.merge(prices_update,indicator = True, how='left').loc[lambda x : x['_merge']!='both']

,A095570,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490,_merge
1496,4290.0,12050,32650,5960.0,187500.0,6990,234000,78100,32450,141500,...,8690,42600.0,28200,377300.0,5790,4575,8520.0,4525,25300.0,left_only
1497,4230.0,11750,33000,5990.0,193500.0,7000,235000,79300,32700,145500,...,8750,42800.0,28250,377700.0,5820,4715,8520.0,4525,26400.0,left_only
1498,4230.0,11450,32200,5940.0,191500.0,6850,233500,77900,31500,144500,...,8780,42450.0,27950,369100.0,5840,4690,8520.0,4515,24950.0,left_only
1499,4235.0,11250,31750,5940.0,192500.0,6910,238500,79900,31350,145000,...,8650,43200.0,28350,367700.0,7370,4620,8520.0,4485,23200.0,left_only


In [205]:
prices_update.merge(prices,indicator = True, left_index=True, right_index=True, how='outer').loc[lambda x : x['_merge']!='both']

,A095570_x,A068400_x,A006840_x,A027410_x,A282330_x,A138930_x,A001460_x,A001040_x,A079160_x,A000120_x,...,A065510_y,A243070_y,A084110_y,A145020_y,A024060_y,A010240_y,A189980_y,A037440_y,A238490_y,_merge
2019-10-14,4220.0,11200.0,32750.0,6080.0,195500.0,7080.0,244000.0,82100.0,31650.0,145500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2019-10-15,4215.0,11050.0,33650.0,6120.0,195500.0,7020.0,241500.0,83100.0,32400.0,144500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2019-10-16,4235.0,11000.0,34400.0,6210.0,191500.0,7120.0,240500.0,84000.0,33200.0,144500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2019-10-17,4280.0,10900.0,33750.0,6220.0,194500.0,7180.0,239500.0,83300.0,33150.0,149000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2019-10-18,4325.0,10800.0,33250.0,6210.0,192500.0,7180.0,239500.0,84700.0,35000.0,155500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2019-10-21,4245.0,10800.0,33450.0,6130.0,190500.0,7220.0,237500.0,84800.0,35000.0,157000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
